<img alt="sudoku-img" src="https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQmJshvMQUu37mCVrVCgYgrk5s9STJPH6JTKj0KEabiNcpgGzDP" width="500">

This Kernel has the objective to present some deterministic approaches to solve a Sudoku puzzle. Two approaches will be used:

* Constraint Programming
* Integer Programming

For both approaches, The [OR Tools](https://developers.google.com/optimization) lib will be used.

OR-Tools is an open source software suite for optimization, tuned for tackling the world's toughest problems in vehicle routing, flows, linear programming, **integer programming and constraint programming**.

# Sudoku Puzzle

Sudoku is a mathematical game that was invented in the late 1970s, became popular in Japan in the 1980s and became known internationally in 2005 when numerous newspapers began publishing it in their hobbies section. 

The objective of the game is to place numbers from 1 to 9 in each of the empty cells in a 9x9 grid, consisting of 3x3 subgrades called regions. The puzzle contains some initial clues, which are numbers inserted into some cells, to allow an induction or deduction of numbers into empty cells. Each column, row, and region can only have a number from each of 1 through 9.

For instance, given the initial Sudoku state:
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/Sudoku-by-L2G-20050714.svg/1024px-Sudoku-by-L2G-20050714.svg.png" width="300">

The final state must be:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/Sudoku-by-L2G-20050714_solution.svg/1024px-Sudoku-by-L2G-20050714_solution.svg.png" width="300">

# Loading sample

The 9 Million Sudoku Puzzle dataset will be used. One puzzle sample for further explanations.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

sudoku = pd.read_csv("../input/sudoku/sudoku.csv") # Loading puzzles from csv
sample = sudoku.loc[2020] # row 2020
sample

The puzzle column shows the initial sudoku state (with missing pieces as 0) and the solution shows the same puzzle solved.

The format is encoded as a single string. I will now create functions to decode it (and encode it back) to a 2d integer matrix.

In [ ]:
def decode_sudoku(sample: str) -> np.matrix:
    '''Transform an encoded puzzle into an integer matrix.'''
    return np.matrix([np.array(list(sample[i:i+9])).astype(np.int) for i in range(0, len(sample), 9)])

decoded_puzzle = decode_sudoku(sample['puzzle'])
decoded_puzzle

In [ ]:
def encode_sudoku(sudoku: np.matrix) -> str:
    '''Transform an integer matrix into an encoded string'''
    return ''.join([''.join(list(r.astype(str))) for r in np.asarray(sudoku)])

encoded_puzzle = encode_sudoku(decoded_puzzle)

assert encoded_puzzle == sample['puzzle'] # must be true, since the same puzzle was decoded and encoded
encoded_puzzle

# Constraint Programming

**Constraint Programming (CP)** is a paradigm for solving combinatorial problems that draws on a wide range of techniques from artificial intelligence, computer science, and operations research. In Constraint Programming, users declaratively state the constraints on the feasible solutions for a set of decision variables. Constraints differ from the common primitives of imperative programming languages in that they do not specify a step or sequence of steps to execute, but rather the properties of a solution to be found. In additions to constraints, users also need to specify a method to solve these constraints. This typically draws upon standard methods like chronological backtracking and constraint propagation, but may use customized code like a problem specific branching heuristic.

Constraint Programming is based on feasibility (finding a feasible solution) rather than optimization (finding an optimal solution) and focuses on the constraints and variables rather than the objective function. In fact, a CP problem may not even have an objective function — the goal may simply be to narrow down a vary large set of possible solutions to a more manageable subset by adding constraints to the problem. However, CP can be used to solve standard optimization problems, which have an objective function, by simply comparing the value of the objective for all feasible solutions. See The Job shop problem for an example of this.

## Modeling a Sudoku Solver with OR Tools Constraint Programming Solver

To start modeling a Sudoku Solver with OR Tools Constraint Programming, the following steps must be executed:

1. To start modeling with OR Tools is to create a model with `cp_model.CpModel()`;
2. Create a grid of variables. Every variables represents a cell of the sudoku solver, this means that every variable must be an integer value and vary from 1 to 9. That been said a matrix $x$ is created, where *known* values (of the initial puzzle) are copied to the matrix and the *unknown* values are initialized as integer variables;
3. Define the *rows*, *columns* and regions *constraints*. OR Tools have a Global Constraint (GC) that declare that a set of variables must have different values. To use the GC, the function `model.AddAllDifferent` must be used;
4. Create a solver using `cp_model.CpSolver()`;
5. Solve model using `solver.Solve(model)`;
6. After model is solved, use `solver.Value` to show the final value of all variables on the $x$ matrix.

In [ ]:
from ortools.sat.python import cp_model

def solve_with_cp(grid: np.matrix) -> (np.matrix, float):
    '''Solve Sudoku instance (np.matrix) with CP modeling. Returns a tuple with the resulting matrix and the execution time in seconds.'''
    assert grid.shape == (9,9)
    
    grid_size = 9
    region_size = 3 #np.sqrt(grid_size).astype(np.int)
    model = cp_model.CpModel() # Step 1

    # Begin of Step2: Create and initialize variables.
    x = {}
    for i in range(grid_size):
        for j in range(grid_size):
            if grid[i, j] != 0:
                x[i, j] = grid[i, j] # Initial values (values already defined on the puzzle).
            else:
                x[i, j] = model.NewIntVar(1, grid_size, 'x[{},{}]'.format(i,j) ) # Values to be found (variyng from 1 to 9).
    # End of Step 2.

    # Begin of Step3: Values constraints.
    # AllDifferent on rows, to declare that all elements of all rows must be different.
    for i in range(grid_size):
        model.AddAllDifferent([x[i, j] for j in range(grid_size)])

    # AllDifferent on columns, to declare that all elements of all columns must be different.
    for j in range(grid_size):
        model.AddAllDifferent([x[i, j] for i in range(grid_size)])

    # AllDifferent on regions, to declare that all elements of all regions must be different.
    for row_idx in range(0, grid_size, region_size):
        for col_idx in range(0, grid_size, region_size):
            model.AddAllDifferent([x[row_idx + i, j] for j in range(col_idx, (col_idx + region_size)) for i in range(region_size)])
    # End of Step 3.

    solver = cp_model.CpSolver() # Step 4
    start = datetime.now()
    status = solver.Solve(model) # Step 5
    exec_time = datetime.now() - start
    result = np.zeros((grid_size, grid_size)).astype(np.int)

    # Begin of Step 6: Getting values defined by the solver
    if status == cp_model.FEASIBLE:
        for i in range(grid_size):
            for j in range(grid_size):
                result[i,j] = int(solver.Value(x[i,j]))
    else:
        raise Exception('Unfeasible Sudoku')
    # End of Step 6

    return result, exec_time.total_seconds()

res, _ = solve_with_cp(decoded_puzzle)
cp_solution = encode_sudoku(res) 

assert cp_solution == sample['solution'] # must show the same solution for the puzzle found on the dataset
res

# Integer Programming

## Linear Programming

**Linear programming (LP)**, also called linear optimization, is a method to achieve the best outcome (such as maximum profit or lowest cost) in a mathematical model whose requirements are represented by linear relationships. Linear programming is a special case of mathematical programming (also known as mathematical optimization).

More formally, linear programming is a technique for the optimization of a linear objective function, subject to linear equality and linear inequality constraints. Its feasible region is a convex polytope, which is a set defined as the intersection of finitely many half spaces, each of which is defined by a linear inequality. Its objective function is a real-valued affine (linear) function defined on this polyhedron. A linear programming algorithm finds a point in the polytope where this function has the smallest (or largest) value if such a point exists.

Linear programs are problems that can be expressed in canonical form as

*Maximize*   $c^t \times x$  
*Subject to* $A \times x \leq b$  
*And*        $x \geq 0$

where $x$ represents the vector of variables (to be determined), $c$ and $b$ are vectors of (known) coefficients, $A$ is a (known) matrix of coefficients, and $(\cdot)^T$ is the matrix transpose. The expression to be maximized or minimized is called the objective function ($c^t \times x$ in this case). The inequalities $A \times x \leq b$ and $x \geq 0$ are the constraints which specify a convex polytope over which the objective function is to be optimized. In this context, two vectors are comparable when they have the same dimensions. If every entry in the first is less-than or equal-to the corresponding entry in the second, then it can be said that the first vector is less-than or equal-to the second vector.

## Mixed Integer Programming

An Integer Programming problem is a special form of Linear Programming in which some or all of the variables are restricted to be integers. In many settings the term refers to integer linear programming (ILP), in which the objective function and the constraints (other than the integer constraints) are linear.

Integer programming is NP-complete. In particular, the special case of 0-1 integer linear programming, in which unknowns are binary, and only the restrictions must be satisfied.

If some decision variables are not discrete the problem is known as a mixed-integer programming (MIP) problem.

For instance, the following model:

Maximize $y$  
Subject to:  
$-x + y \leq 1$  
$3x + 2y \leq 12$  
$2x + 3y \leq 12$  
$x, y \geq 0$  
$x, y \in \mathbb{Z}$

Can be plotted like the image below, where the blue lines are the linear bounds, but $x$ and $y$ **can't** assume any of these values, since they're not integers. However, the red dots are under the blue area and are integer values, these are the values $x$ and $y$ **can** assume. Since this model tries do maximize $y$, this means that $y$ must be 2 and $x$ can be 1 or 2. Both this values will maximize $y$ and satisfy all the declared constraints.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/06/IP_polytope_with_LP_relaxation.svg/350px-IP_polytope_with_LP_relaxation.svg.png"/>


## Modeling a Sudoku Solver with OR Tools Integer Programming Solver

For this IP formulation there is no solution that is better than another, only a feasible solution matters, the one which satisfies all constraints. Therefore, this formulation will not require an objective function. That been said, this formulation is quite similar to the CP model, but with a subtle difference: instead of a 2D matrix with integer variables (varying from 1 to 9), a binary representation of a 3D matrix will be used.

For instance, the simple attribution $x[0,3] = 2$ is translated to 9 attributions like: $x[0,3,0] = 0$, $x[0,3,1] = 1$, $x[0,3,2] = 0$, ..., $x[0,3,8] = 0$. On this representation the model must assure that only one possible value for a cell is valid (1) while the others must be invalid (0), therefore the additional constraint must be added to the model:

$\sum_{0}^{k-1} x[i, j, k] = 1, \forall i, j$ (*)

To start modeling a Sudoku Solver with OR Tools Integer Programming, the following steps must be executed:

1. To start modeling with OR Tools is to create an instance of the solver with `pywraplp.Solver(...)`. On this example, the CBC MIP Solver is used;
2. Create a grid of variables. Every variables represents a possible value of a cell of the sudoku, this means that every variable must be a boolean value and vary from 0 to 1. That been said a matrix $x$ and all values are initialized as boolean variables;
3. Add constraints to fetch all initial values on the *known* values;
4. Add constraint to assure that only one possible value for a cell is valid (*);
5. Define the *rows*, *columns* and regions *constraints*.
6. Solve model using `solver.Solve()`;
7. After model is solved, use `x[i, j, k].solution_value()` to show the final value of all variables on the $x$ matrix and convert it from boolean to integer.

In [ ]:
from ortools.linear_solver import pywraplp

def solve_with_ip(grid: np.ndarray) -> (np.ndarray, float):
    '''Solve Sudoku instance (np.matrix) with IP modeling. Returns a tuple with the resulting matrix and the execution time in seconds.'''
    assert grid.shape == (9,9)
    
    grid_size = 9
    cell_size = 3 #np.sqrt(grid_size).astype(np.int)
    solver = pywraplp.Solver('Sudoku Solver', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING) # Step 1

    # Begin of Step2: Create variables.
    x = {}
    for i in range(grid_size):
        for j in range(grid_size):
            # Initial values.
            for k in range(grid_size):
                x[i, j, k] = solver.BoolVar('x[%i,%i,%i]' % (i, j, k))
    # End of Step2
    
    # Begin of Step3: Initialize variables in case of known (defined) values.
    for i in range(grid_size):
        for j in range(grid_size):
            defined = grid[i, j] != 0
            if defined:
                solver.Add(x[i,j,grid[i, j]-1] == 1)
    # End of Step3
    
    # Begin of Step4: Initialize variables in case of known (defined) values. 
    # All bins of a cell must have sum equals to 1
    for i in range(grid_size):
        for j in range(grid_size):
            solver.Add(solver.Sum([x[i, j, k] for k in range(grid_size)]) == 1)
    # End of Step4

    # Begin of Step5: Create variables.
    for k in range(grid_size):
        # AllDifferent on rows.
        for i in range(grid_size):
            solver.Add(solver.Sum([x[i, j, k] for j in range(grid_size)]) == 1)

        # AllDifferent on columns.
        for j in range(grid_size):
            solver.Add(solver.Sum([x[i, j, k] for i in range(grid_size)]) == 1)

        # AllDifferent on regions.
        for row_idx in range(0, grid_size, cell_size):
            for col_idx in range(0, grid_size, cell_size):
                solver.Add(solver.Sum([x[row_idx + i, j, k] for j in range(col_idx, (col_idx + cell_size)) for i in range(cell_size)]) == 1)
    # End of Step5

    # Solve and print out the solution.
    start = datetime.now()
    status = solver.Solve() # Step 6
    exec_time = datetime.now() - start
    statusdict = {0:'OPTIMAL', 1:'FEASIBLE', 2:'INFEASIBLE', 3:'UNBOUNDED', 
                  4:'ABNORMAL', 5:'MODEL_INVALID', 6:'NOT_SOLVED'}
    
    result = np.zeros((grid_size, grid_size)).astype(np.int)
    if status == pywraplp.Solver.OPTIMAL:
        for i in range(grid_size):
            for j in range(grid_size):
                result[i,j] = sum((k + 1) * int(x[i, j, k].solution_value()) for k in range(grid_size))
    else:
        raise Exception('Unfeasible Sudoku: {}'.format(statusdict[status]))

    return result, exec_time.total_seconds()

res, _ = solve_with_ip(decoded_puzzle)
ip_solution = encode_sudoku(res) 

assert ip_solution == sample['solution'] # must show the same solution for the puzzle found on the dataset
res

In [ ]:
def solve_sudoku(instance: np.matrix, solver: str = 'ip') -> (np.matrix, float):
    if solver == 'ip':
        return solve_with_ip(instance)
    elif solver == 'cp':
        return solve_with_cp(instance)
    else:
        raise Exception('Unknown solver: {}'.format(solver))

solve_sudoku(decode_sudoku(sample['puzzle']))

# Comparing approaches

Now is time to compare how both approaches perform. To achieve this goal both methods will be used to solve a sample of size 1000 collected from the original dataset. Since both methods also returns the execution time in seconds, this data will be registered for further analysis.

The code below save the data of execution time on the `performance_df` dataframe.

In [ ]:
from tqdm.notebook import tqdm

sample_size = 1000
seed = 2020
ip_exec_time = []
cp_exec_time = []

for index, row in tqdm(sudoku.sample(sample_size, random_state=seed).iterrows()):
    res, exec_time = solve_sudoku(decode_sudoku(row.puzzle), 'cp') # Solving with CP
    assert encode_sudoku(res) == row.solution # Assert if result equals to the expected solution
    cp_exec_time += [exec_time] # Register the solver execution time
    
    res, exec_time = solve_sudoku(decode_sudoku(row.puzzle), 'ip') # Solving with IP
    assert encode_sudoku(res) == row.solution # Assert if result equals to the expected solution
    ip_exec_time += [exec_time] # Register the solver execution time
    
performance_df = pd.DataFrame({'IP' : ip_exec_time, 'CP' : cp_exec_time})
performance_df.head()

In [ ]:
performance_df.plot.hist(subplots=True, figsize=(12, 4), layout=(1,2))

As can be seen, both methods are really fast on solving sudoku, however CP has showed consistently better execution time than the IP approach.

## References 

* https://www.kaggle.com/danielmartinezb/irregular-6x6-sudoku-a-search
* https://en.wikipedia.org/wiki/Constraint_programming
* https://pt.wikipedia.org/wiki/Sudoku
* https://developers.google.com/optimization
* https://towardsdatascience.com/using-integer-linear-programming-to-solve-sudoku-puzzles-15e9d2a70baa
* https://en.wikipedia.org/wiki/Linear_programming
* https://en.wikipedia.org/wiki/Integer_programming